#  app


In [7]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import tkinter as tk
from PIL import Image, ImageTk


image_paths = []
labels = []

folder_path = './images/'
csv_file_path = 'pollen_data.csv'

data = pd.read_csv(csv_file_path)

image_paths = [os.path.join(folder_path, filename) for filename in data['filename']]
labels = data['pollen_carrying'].values.astype(int)

def load_images():
    honeybee_images = []
    non_honeybee_images = []

    for path, label in zip(image_paths, labels):
        img = cv2.imread(path)

        if label == 1:
            honeybee_images.append(img)
        else:
            non_honeybee_images.append(img)

    return honeybee_images, non_honeybee_images

def apply_color_space_transform(images, sigma_h=0.1, sigma_s=0.8, sigma_v=0.3, sigma_smooth_h=0.05, sigma_smooth_s=0.05, sigma_smooth_v=0.8):
    transformed_images = []

    for img in images:
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # Apply Gaussian model to HSV channels
        img_hsv[:,:,0] += int(sigma_h * 255)
        img_hsv[:,:,1] += int(sigma_s * 255)
        img_hsv[:,:,2] += int(sigma_v * 255)
        img_hsv = np.clip(img_hsv, 0, 255).astype(np.uint8)

        # Apply Gaussian smoothing to the HSV channels
        img_hsv[:,:,0] = cv2.GaussianBlur(img_hsv[:,:,0], (0, 0), sigma_smooth_h)
        img_hsv[:,:,1] = cv2.GaussianBlur(img_hsv[:,:,1], (0, 0), sigma_smooth_s)
        img_hsv[:,:,2] = cv2.GaussianBlur(img_hsv[:,:,2], (0, 0), sigma_smooth_v)

        transformed_img = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
        transformed_images.append(transformed_img)

    return transformed_images

def resize_images(images, width, height):
    resized_images = []

    for img in images:
        resized_img = cv2.resize(img, (width, height))
        resized_images.append(resized_img)

    return resized_images

def extract_color_histogram_local(images):
    histograms = []

    for img in images:
        # Chuyển đổi hình ảnh sang không gian màu HSV
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # Chia hình ảnh thành các ô lưới và tính histogram cho mỗi ô lưới
        num_grid_x = 14
        num_grid_y = 7
        grid_size_x = hsv.shape[1] // num_grid_x
        grid_size_y = hsv.shape[0] // num_grid_y
        hist_features = []

        for y in range(num_grid_y):
            for x in range(num_grid_x):
                grid = hsv[y*grid_size_y:(y+1)*grid_size_y, x*grid_size_x:(x+1)*grid_size_x, :]
                hist_h = cv2.calcHist([grid], [0], None, [16], [0, 180])  # Hue histogram
                hist_s = cv2.calcHist([grid], [1], None, [8], [0, 256])   # Saturation histogram
                hist_v = cv2.calcHist([grid], [2], None, [8], [0, 256])   # Value histogram

                # Chuẩn hóa histogram bằng cách chia cho số lượng pixel trong mỗi ô lưới
                hist_h /= grid_size_x * grid_size_y
                hist_s /= grid_size_x * grid_size_y
                hist_v /= grid_size_x * grid_size_y

                # Ghép các histogram thành một vector đặc trưng
                hist_features.extend(hist_h.flatten())
                hist_features.extend(hist_s.flatten())
                hist_features.extend(hist_v.flatten())

        histograms.append(hist_features)

    return histograms

def prepare_data(honeybee_images, non_honeybee_images):
    honeybee_labels = np.ones(len(honeybee_images))
    non_honeybee_labels = np.zeros(len(non_honeybee_images))

    images = honeybee_images + non_honeybee_images
    labels = np.concatenate([honeybee_labels, non_honeybee_labels])

    # Apply color space transformation
    transformed_images = apply_color_space_transform(images)

    # Resize images
    resized_images = resize_images(transformed_images, 150, 90)

    # Extract color histogram local features
    hist_features = extract_color_histogram_local(resized_images)

    X_train, X_test, y_train, y_test = train_test_split(hist_features, labels, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

def train_svm(X_train, y_train):
    svm = SVC(kernel='rbf', C=1.0, gamma='scale')
    svm.fit(X_train, y_train)
    return svm

def train_knn(X_train, y_train):
    knn = KNeighborsClassifier()
    knn.fit(X_train, y_train)
    return knn

def train_nb(X_train, y_train):
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    return nb

def evaluate_model(model, X_test, y_test):
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    confusion_mat = confusion_matrix(y_test, y_pred)
    FP = confusion_mat[0, 1]
    TP = confusion_mat[1, 1]
    FN = confusion_mat[1, 0]
    TN = confusion_mat[0, 0]
    return accuracy, TP, FP, TN, FN

def classify_image(image_path, model):
    transformed_image = apply_color_space_transform([cv2.imread(image_path)])
    resized_image = resize_images(transformed_image, 150, 90)
    hist_feature = extract_color_histogram_local(resized_image)
    predicted_label = model.predict(hist_feature)
    return "ong mang phan" if predicted_label[0] == 1 else "khong phai ong mang phan"

def display_random_image():
    global svm_model

    # Chọn ngẫu nhiên một ảnh từ thư mục "./images/"
    random_image_path = random.choice(image_paths)

    # Phân loại ảnh dựa vào model đã train
    result = classify_image(random_image_path, svm_model)

    # Hiển thị ảnh gốc
    original_img = Image.open(random_image_path)
    original_img = original_img.resize((300, 180))  # Resize ảnh để phù hợp với khung hiển thị
    original_img_tk = ImageTk.PhotoImage(original_img)
    rgb_label.config(image=original_img_tk)
    rgb_label.image = original_img_tk

    # Hiển thị ảnh dạng HSV
    hsv_img = cv2.cvtColor(cv2.imread(random_image_path), cv2.COLOR_BGR2HSV)
    hsv_img = Image.fromarray(cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB))
    hsv_img = hsv_img.resize((300, 180))  # Resize ảnh để phù hợp với khung hiển thị
    hsv_img_tk = ImageTk.PhotoImage(hsv_img)
    hsv_label.config(image=hsv_img_tk)
    hsv_label.image = hsv_img_tk

    # Hiển thị kết quả phân loại
    result_label.config(text=f"Ảnh {random_image_path} được phân loại là: {result}")

# Tạo giao diện Tkinter
root = tk.Tk()
root.title("Histogram và Hình ảnh tương ứng")

# Nút Hiển thị ngẫu nhiên ảnh
random_button = tk.Button(root, text="Hiển thị ngẫu nhiên ảnh", command=display_random_image)
random_button.pack(pady=10)

# Khung hiển thị histogram
histogram_frame = tk.Frame(root)
histogram_frame.pack()

# Khung hiển thị ảnh gốc
rgb_frame = tk.Frame(root)
rgb_frame.pack(side=tk.LEFT)

# Khung hiển thị ảnh dạng HSV
hsv_frame = tk.Frame(root)
hsv_frame.pack(side=tk.LEFT)

# Khung hiển thị kết quả phân loại
result_label = tk.Label(root, text="", font=("Helvetica", 14))
result_label.pack(pady=10)

# Khởi tạo ảnh mặc định cho hai khung hiển thị ảnh
default_image = Image.new("RGB", (300, 180), color=(255, 255, 255))
default_image_tk = ImageTk.PhotoImage(default_image)

rgb_label = tk.Label(rgb_frame, image=default_image_tk)
rgb_label.pack(padx=10)

hsv_label = tk.Label(hsv_frame, image=default_image_tk)
hsv_label.pack(padx=10)

# Chạy giao diện
root.mainloop()

# Prepare data
honeybee_images, non_honeybee_images = load_images()
X_train, X_test, y_train, y_test = prepare_data(honeybee_images, non_honeybee_images)

# Train SVM model
svm_model = train_svm(X_train, y_train)

# Print the results
print("Results:")
svm_accuracy, svm_TP, svm_FP, svm_TN, svm_FN = evaluate_model(svm_model, X_test, y_test)
print("SVM:")
print(f"Accuracy: {svm_accuracy}")
print(f"True Positive: {svm_TP}")
print(f"False Positive: {svm_FP}")
print(f"True Negative: {svm_TN}")
print(f"False Negative: {svm_FN}")


Results:
SVM:
Accuracy: 0.8811188811188811
True Positive: 76
False Positive: 10
True Negative: 50
False Negative: 7


In [ ]:
#test
def apply_color_space_transform(images, sigma_h=0.1, sigma_s=0.8, sigma_v=0.3, sigma_smooth_h=0.05, sigma_smooth_s=0.05, sigma_smooth_v=0.8):
    transformed_images = []
    
    for img in images:
        img_hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        
        # Apply Gaussian model to HSV channels
        img_hsv[:,:,0] += int(sigma_h * 255)
        img_hsv[:,:,1] += int(sigma_s * 255)
        img_hsv[:,:,2] += int(sigma_v * 255)
        img_hsv = np.clip(img_hsv, 0, 255).astype(np.uint8)
        
        # Apply Gaussian smoothing to the HSV channels
        img_hsv[:,:,0] = cv2.GaussianBlur(img_hsv[:,:,0], (0, 0), sigma_smooth_h)
        img_hsv[:,:,1] = cv2.GaussianBlur(img_hsv[:,:,1], (0, 0), sigma_smooth_s)
        img_hsv[:,:,2] = cv2.GaussianBlur(img_hsv[:,:,2], (0, 0), sigma_smooth_v)
        
        transformed_img = cv2.cvtColor(img_hsv, cv2.COLOR_HSV2BGR)
        transformed_images.append(transformed_img)
            
    return transformed_images
import cv2
import numpy as np
import os
import random

# Đường dẫn thư mục chứa ảnh
folder_path = './images/'

# Lấy danh sách tất cả các tệp ảnh trong thư mục
image_files = os.listdir(folder_path)

# Chọn ngẫu nhiên một ảnh từ danh sách
random_image_file = random.choice(image_files)

# Đường dẫn đầy đủ của ảnh được chọn ngẫu nhiên
random_image_path = os.path.join(folder_path, random_image_file)

# Đọc ảnh từ đường dẫn được chọn
img = cv2.imread("./images/P14842-230r.jpg")

# Áp dụng hàm apply_color_space_transform để biến đổi ảnh
transformed_images = apply_color_space_transform([img])

# Hiển thị ảnh gốc và ảnh sau khi qua hàm apply_color_space_transform
cv2.imshow('Original Image', img)
cv2.imshow('Transformed Image', transformed_images[0])

# Chờ bấm phím bất kỳ để đóng cửa sổ hiển thị ảnh
cv2.waitKey(0)
cv2.destroyAllWindows()


In [5]:
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import os
import pandas as pd
import matplotlib.pyplot as plt
import random
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk


def display_random_image():
    # Chọn ngẫu nhiên một ảnh từ thư mục "./images/"
    random_image_path = random.choice(image_paths)

    # Phân loại ảnh dựa vào model đã train
    result = classify_image(random_image_path, model)

    # Hiển thị ảnh gốc
    original_img = Image.open(random_image_path)
    original_img = original_img.resize((300, 180))  # Resize ảnh để phù hợp với khung hiển thị
    original_img_tk = ImageTk.PhotoImage(original_img)
    rgb_label.config(image=original_img_tk)
    rgb_label.image = original_img_tk

    # Hiển thị ảnh dạng HSV
    hsv_img = cv2.cvtColor(cv2.imread(random_image_path), cv2.COLOR_BGR2HSV)
    hsv_img = Image.fromarray(cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB))
    hsv_img = hsv_img.resize((300, 180))  # Resize ảnh để phù hợp với khung hiển thị
    hsv_img_tk = ImageTk.PhotoImage(hsv_img)
    hsv_label.config(image=hsv_img_tk)
    hsv_label.image = hsv_img_tk

    # Hiển thị kết quả phân loại
    result_label.config(text=f"Ảnh {random_image_path} được phân loại là: {result}")

# Tạo giao diện Tkinter
root = tk.Tk()
root.title("Histogram và Hình ảnh tương ứng")

# Nút Hiển thị ngẫu nhiên ảnh
random_button = tk.Button(root, text="Hiển thị ngẫu nhiên ảnh", command=display_random_image)
random_button.pack(pady=10)

# Khung hiển thị histogram
histogram_frame = tk.Frame(root)
histogram_frame.pack()

# Khung hiển thị ảnh gốc
rgb_frame = tk.Frame(root)
rgb_frame.pack(side=tk.LEFT)

# Khung hiển thị ảnh dạng HSV
hsv_frame = tk.Frame(root)
hsv_frame.pack(side=tk.LEFT)

# Khung hiển thị kết quả phân loại
result_label = tk.Label(root, text="", font=("Helvetica", 14))
result_label.pack(pady=10)

# Khởi tạo ảnh mặc định cho hai khung hiển thị ảnh
default_image = Image.new("RGB", (300, 180), color=(255, 255, 255))
default_image_tk = ImageTk.PhotoImage(default_image)

rgb_label = tk.Label(rgb_frame, image=default_image_tk)
rgb_label.pack(padx=10)

hsv_label = tk.Label(hsv_frame, image=default_image_tk)
hsv_label.pack(padx=10)

# Chạy giao diện
root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\MINH.NV193012\AppData\Local\anaconda3\lib\tkinter\__init__.py", line 1921, in __call__
    return self.func(*args)
  File "C:\Users\MINH.NV193012\AppData\Local\Temp\ipykernel_17096\3387022390.py", line 22, in display_random_image
    result = classify_image(random_image_path, model)
NameError: name 'model' is not defined
